# Морфологические анализаторы на трансдьюсерах

Мы уже видели морфологические анализаторы для разных языков (pymorphy, Mystem, spacy, stanza и др.)

А делить на морфемы-то как?

Хотим глоссировать!



## Если мало данных, можно использовать правила

Исходить из внешнего вида слова, думать о значении морфем.

#### Отступление: какие бывают языки по тому, как устроено слово?

- Аналитические
- Синтетические
 - Флективные
 - Агглютинативные

Для каких проще написать правила?

### Английский пример (потому что знаем английский)

Можно вручную расписывать, "если на конце s, а корень похож на существительное, то это существительное с показателем множественного числа, а если корень похож на глагол, это 3е лицо ед.ч." И так условия для каждой морфемы. Но есть некоторые готовые способы записи правил.

# Трансдьюсеры
Например [HFST](https://github.com/apertium/lexd/blob/main/Usage.md) (Helsinki Finite-State Transducer)

In [1]:
!curl -sS https://apertium.projectjj.com/apt/install-release.sh | sudo bash
!apt install apertium-all-dev lexd

Cleaning up old install, if any...
Determining Debian/Ubuntu codename...
Found evidence of jammy...
Settling for jammy - enabling the Apertium release repo...
Installing Apertium GnuPG key to /etc/apt/trusted.gpg.d/apertium.gpg
Installing package override to /etc/apt/preferences.d/apertium.pref
Creating /etc/apt/sources.list.d/apertium.list
Running apt-get update...
All done - enjoy the packages! If you just want all core tools, do: sudo apt-get install apertium-all-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apertium apertium-anaphora apertium-dev apertium-eval-translator apertium-get apertium-lex-tools
  apertium-lex-tools-dev apertium-recursive apertium-regtest apertium-separable cg3 cg3-dev gawk
  hfst libapertium-lex-tools1 libapertium3 libcg3-1 libcg3-dev libfoma0 libfst22 libhfst-dev
  libhfst55 libirstlm1 liblttoolbox3 libutfcpp-dev libxml2-utils libzip4 lttoolbox ltt

Это не питон, что это за язык? То, что после восклицательного знака - команды Linux (линуксовские), команды, которые работают в терминале

In [ ]:
%%writefile eng.lexd
PATTERNS
VerbRoot VerbInfl

LEXICON VerbRoot
sing
walk

LEXICON VerbInfl
<v><pres>:
<v><pres><p3><sg>:s

Writing eng.lexd


In [ ]:
!lexd eng.lexd > eng.generator.att
!lt-comp rl eng.generator.att eng.analyser.bin
! echo 'sings' | lt-proc eng.analyser.bin  # даёт разбор слова

main@standard 13 14
^sings/sing<v><pres><p3><sg>$


```
- Где деление на морфемы?
- Будет.
```

Если вы поняли логику, научитесь разбирать ещё слова cat, cats, dog, dogs.

In [ ]:
%%writefile eng.lexd
PATTERNS
VerbRoot VerbInfl
NounRoot NounInfl

LEXICON VerbRoot
sing
walk
dance

LEXICON VerbInfl
<v><pres>:
<v><pres><p3><sg>:s

LEXICON NounRoot
cat
dog

LEXICON NounInfl
<n><sg>:
<n><pl>:s


Overwriting eng.lexd


In [ ]:
# пересобираем генератор и анализатор
!lexd eng.lexd > eng.generator.att
!lt-comp rl eng.generator.att eng.analyser.bin

main@standard 22 27


In [ ]:
! echo 'dogs' | lt-proc eng.analyser.bin

^dogs/dog<n><pl>$


In [ ]:
! lexd eng.lexd | hfst-txt2fst -o eng.generator.hfst
! hfst-fst2strings eng.generator.hfst  # генерирует все возможные формы

sing<v><pres>:sing
sing<v><pres><p3><sg>:sings
walk<v><pres>:walk
walk<v><pres><p3><sg>:walks
dance<v><pres>:dance
dance<v><pres><p3><sg>:dances
dog<n><sg>:dog
dog<n><pl>:dogs
cat<n><sg>:cat
cat<n><pl>:cats


### А что происходит, что такое Finite-State Transducer?

Finite-State Transducer - конечный преобразовывающий автомат

Понятнее? :)

Автомат - математическая модель, они бывают разные, поэтому давайте сразу про конечные автоматы


Конечный атомат состоит из:
- "алфавита" состояний
- входного алфавита
- переходов между состояниями
- нального состояния
- конечных состояний

Можно представить в виде
- графа
- матрицы
- БНФ (Бэкусовская нормальная форма или Бэкуса-Науэра форма)

![картинка тут](https://raw.githubusercontent.com/tbkazakova/drafts/main/auto.png)

ε – пустой символ

Бывают автоматы
- распознающие — определяют соответствие входной цепочки конечному автомату
- преобразующие — производят определенные действия по входной цепочке

Наш автомат преобразующий: он ходит по слову и по частям заменяет слово на <морфологические теги>


Подвид КА (конечных автоматов):
- стохастический КА — каждому переходу приписана его вероятность

(в lexd можно это прописать, ведь есть более вероятные разборы и менее)

### Возвращаемся к HFST
На самом деле в HFST две больших части, два формализма [lexd](https://github.com/apertium/lexd/blob/main/Usage.md) и twolc (two-level compiler).

twolc нужен для морфонологии. Порой одна глубинная форма реализуется по-разному. (примеры из русского?)

Как это записать, если не хочется указывать в лексиконе алломорфы разными сущностями?

Вот для этого и нужен twolc. two-level compiler: на первом уровне даём "глубинные формы", на втором расписываем их поверхностные реализации.


### Английский (правила для s во множественном числе)

Иногда показатель PL выглядит как *-es*, а не *-s*. В каких случаях?

Как вы думаете, какая глубинная форма у этого показателя? Почему?

In [18]:
%%writefile eng.lexd
PATTERNS
NounRoot NounInfl

LEXICON NounRoot
dog
cat
bus
fox
watch
wish
fez

LEXICON NounInfl
<n><sg>:
<n><pl>:{S}

Overwriting eng.lexd


In [19]:
!lexd eng.lexd > eng.generator.att
!lt-comp rl eng.generator.att eng.analyser.bin
! echo 'dogs' | lt-proc eng.analyser.bin
! echo 'dog\{S\}' | lt-proc eng.analyser.bin

main@standard 21 27
^dogs/*dogs$
^dog\{S\}/dog<n><pl>$


In [20]:
! lexd eng.lexd | hfst-txt2fst -o eng.generator.hfst
! hfst-invert eng.generator.hfst -o eng.analyzer.hfst

In [36]:
%%writefile eng.twol
Alphabet
  a b c d e f g h i j k l m n o p q r s t u v w x y z
  %{S%}:es %{S%}:s
;
Rules
"{S} s _ realisation"
%{S%}:es <=> s|x|z|(ch)|(sh) _ ;

Overwriting eng.twol


In [37]:
! hfst-twolc eng.twol -o eng.twol.hfst
! lexd eng.lexd | hfst-txt2fst -o eng.lexd.hfst
#! hfst-fst2strings eng.lexd.hfst
! hfst-compose-intersect eng.lexd.hfst eng.twol.hfst -o eng.generator.hfst
#! hfst-fst2strings eng.generator.hfst
! hfst-invert eng.generator.hfst -o eng.analyzer.hfst

Reading input from eng.twol.
Writing output to eng.twol.hfst.
Reading alphabet.
Reading rules and compiling their contexts and centers.
Compiling rules.
Storing rules.


In [28]:
a = ! echo "dogs" | hfst-lookup eng.analyzer.hfst
a[2].split()[2]

'dogs+?'

In [24]:
a = ! echo "foxes" | hfst-lookup eng.analyzer.hfst
a[2].split()[2]

'fox<n><pl>'

In [8]:
a = ! echo "buses" | hfst-lookup eng.analyzer.hfst
a[2].split()[2]

'bus<n><pl>'

In [9]:
# а неправильные слова не разбирает
a = ! echo "doges" | hfst-lookup eng.analyzer.hfst
a[2].split()[2]

'doges+?'

In [35]:
! hfst-fst2strings eng.generator.hfst  # генерирует все возможные формы

bus<n><sg>:bus
bus<n><pl>:buses
cat<n><sg>:cat
cat<n><pl>:cates
dog<n><sg>:dog
dog<n><pl>:doges
fez<n><sg>:fez
fez<n><pl>:fezes
fox<n><sg>:fox
fox<n><pl>:foxes
watch<n><sg>:watch
watch<n><pl>:watches
wish<n><sg>:wish
wish<n><pl>:wishes


Можете добавить остальные случаи, когда "es".

### Как сделать сегментацию?

In [45]:
%%writefile eng.lexd
PATTERNS
NounRoot NounInfl

LEXICON NounRoot
dog
cat
bus

LEXICON NounInfl
<n><sg>:
<n><pl>:>{S}

Overwriting eng.lexd


In [39]:
%%writefile eng.twol
Alphabet
  a b c d e f g h i j k l m n o p q r s t u v w x y z
  %{S%}:es %{S%}:s
  %>:0
;
Rules
"{S} s _ realisation"
%{S%}:es <=> s %>: _ ;

Overwriting eng.twol


In [40]:
!lexd eng.lexd > eng.generator.att
!lt-comp rl eng.generator.att eng.analyser.bin
! hfst-twolc eng.twol -o eng.twol.hfst
! lexd eng.lexd | hfst-txt2fst -o eng.lexd.hfst
#! hfst-fst2strings eng.lexd.hfst
! hfst-compose-intersect eng.lexd.hfst eng.twol.hfst -o eng.generator.hfst
#! hfst-fst2strings eng.generator.hfst
! hfst-invert eng.generator.hfst -o eng.analyzer.hfst

main@standard 13 15
Reading input from eng.twol.
Writing output to eng.twol.hfst.
Reading alphabet.
Reading rules and compiling their contexts and centers.
Compiling rules.
Storing rules.


In [41]:
! hfst-fst2strings eng.generator.hfst  # генерирует все возможные формы

bus<n><sg>:bus
bus<n><pl>:buses
cat<n><sg>:cat
cat<n><pl>:cats
dog<n><sg>:dog
dog<n><pl>:dogs


In [42]:
!hfst-invert eng.lexd.hfst -o eng.inv_seg.hfst
!hfst-compose-intersect -1 eng.lexd.hfst -2 eng.twol.hfst | hfst-minimise -o eng-seg.hfst
!hfst-compose -1 eng.inv_seg.hfst -2 eng-seg.hfst -o eng.seg.hfst
!hfst-minimise eng.seg.hfst | hfst-invert | hfst-fst2fst -O -o eng.segmenter.hfst

In [43]:
! hfst-fst2strings eng.segmenter.hfst  # генерирует все возможные формы

bus
buses:bus>{S}
cat
cats:cat>{S}
dog
dogs:dog>{S}


In [47]:
answer_seg = ! echo 'buses' | hfst-lookup eng.segmenter.hfst
answer_gl = ! echo 'buses' | hfst-lookup eng.analyzer.hfst

parts = answer_seg[0].split()[2].replace('>', '-')
gloss = answer_gl[2].split()[2].replace('<n>', '').replace('<', '-').replace('>', '')

parts, gloss

('bus-{S}', 'bus-pl')

Можно ли получить *bus-es*, а не *bus-{S}*?

Можно ли разбирать слова с неизвестным корнем?

Можно ли добавить ранжирование вариантов разбора по вероятности?

- Можно, но я ещё не научилась.


Где взять уже готовые анализаторы?
- Посмотреть репозитории проекта [apertium](https://github.com/apertium). Есть [онлайн версии](https://beta.apertium.org/index.eng.html#analysis?aLang=sah&aQ=%D0%91%D1%83%20%D1%81%D0%B0%D1%85%D0%B0%D0%BB%D1%8B%D1%8B%20%D0%BC%D0%BE%D1%80%D1%84%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D0%B9%20%D1%8B%D1%80%D1%8B%D1%82%D1%8B%D1%8B.), чтобы попробовать.


### Эвенский (глаголы)

Вот [список глосс эвенской экспедиции](https://www.evenlang.ru/evenlang_online/static/files/eve_glosses_draft_20220115_official.pdf)

Научимся разбирать *ur-ri-wu* (уйти-pst-1sg) и *em-ni-wu* (прийти-pst-1sg). r перед n всегда ассимилируется.

Добавьте правила для *it-ti-wu* (увидеть-pst-1sg), *das̆-s̆i-wu* (укрыть-pst-1sg). Посмотрите на таблицу глосс и добавьте оставшиеся варианты реализации <pst>.

Добавим необязательный слот для аспекта перед показателем времени.

Попробуйте добавить парадигму непрошедшего времени.

In [52]:
%%writefile eve.lexd
PATTERNS
VerbStem [<v>:] VerbTencePNum

LEXICON VerbStem
<уйти>:ur
<прийти>:em

PATTERN VerbTencePNum
Verbpst VerbPNumpst

LEXICON Verbpst
<pst>:{R}i

LEXICON VerbPNumpst
<p1><sg>:wu
<p1><sg>:w
<p2><sg>:š
<p3><sg>:n
<p1><pl><inc>:t
<p1><pl><exc>:wun
<p2><pl>:šan
<p3><pl>:tan

Overwriting eve.lexd


In [53]:
!lexd eve.lexd > eve.generator.att
!lt-comp rl eve.generator.att eve.analyser.bin
! echo 'urrin' | lt-proc eve.analyser.bin

main@standard 18 25
^urrin/*urrin$


In [54]:
! lexd eve.lexd | hfst-txt2fst -o eve.generator.hfst
! hfst-invert eve.generator.hfst -o eve.analyzer.hfst

In [55]:
%%writefile eve.twol
Alphabet
  a b c d e f g i j k l m n o p r s t u v w x z ŋ č ž š ə ɨ ɵ γ
  %{R%}:n %{R%}:r
;
Rules
"{R} m _ realisation"
%{R%}:n <=> m _ ;

Overwriting eve.twol


In [56]:
! hfst-twolc eve.twol -o eve.twol.hfst
! lexd eve.lexd | hfst-txt2fst -o eve.lexd.hfst
#! hfst-fst2strings eve.lexd.hfst
! hfst-compose-intersect eve.lexd.hfst eve.twol.hfst -o eve.generator.hfst
#! hfst-fst2strings eve.generator.hfst
! hfst-invert eve.generator.hfst -o eve.analyzer.hfst

Reading input from eve.twol.
Writing output to eve.twol.hfst.
Reading alphabet.
Reading rules and compiling their contexts and centers.
Compiling rules.
Storing rules.


In [57]:
! echo "urrin" | hfst-lookup eve.analyzer.hfst

hfst-lookup: warning: It is not possible to perform fast lookups with OpenFST, std arc, tropical semiring format automata.
Using HFST basic transducer format and performing slow lookups
> urrin	<уйти><v><pst><p3><sg>	0.000000

> 

In [58]:
a = ! echo "emnin" | hfst-lookup eve.analyzer.hfst
a[2].split()[2]

'<прийти><v><pst><p3><sg>'

# Готовое

### Попробуем библиотеку polyglot на основе morfessor (это не трансдьюсеры, это другое)

In [59]:
! pip install polyglot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52558 sha256=69479e0acded4febad189047d7500d629237fced16138ec37682e7eecf9de60e
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot


In [60]:
! pip install icu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.1 MB/s eta 0:00:00


In [61]:
! pip install pyicu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.12-cp310-cp310-linux_x86_64.whl size=1754544 sha256=c2f97a02a83d2033875fcbe535302318da97aeb5a48a5f62f6000319ebf63060
  Stored in directory: /root/.cache/pip/wheels/74/60/95/66d97ac2fdc8be8e526c4254047405fe77feaf064282d1ad07
Successfully built pyicu


In [62]:
! pip install pycld2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycld2: filename=pycld2-0.41-cp310-cp310-linux_x86_64.whl size=9904038 sha256=e2b5d159d5ae354e95d2997b887113caa3fef0fd01fe2bfd57f20aa6958397fe
  Stored in directory: /root/.cache/pip/wheels/be/81/31/240c89c845e008a93d98542325270007de595bfd356eb0b06c
Successfully built pycld2


In [63]:
! pip install morfessor

In [64]:
from polyglot.downloader import downloader
print(downloader.supported_languages_table("morph2"))

  1. Kapampangan                2. Italian                    3. Upper Sorbian            
  4. Sakha                      5. Hindi                      6. French                   
  7. Spanish; Castilian         8. Vietnamese                 9. Arabic                   
 10. Macedonian                11. Pashto, Pushto            12. Bosnian-Croatian-Serbian 
 13. Egyptian Arabic           14. Norwegian Nynorsk         15. Sundanese                
 16. Sicilian                  17. Azerbaijani               18. Bulgarian                
 19. Yoruba                    20. Tajik                     21. Georgian                 
 22. Tatar                     23. Galician                  24. Malagasy                 
 25. Uighur, Uyghur            26. Amharic                   27. Venetian                 
 28. Yiddish                   29. Norwegian                 30. Alemannic                
 31. Estonian                  32. West Flemish              33. Divehi; Dhivehi; Mald... 

In [ ]:
from polyglot.text import Text, Word

In [ ]:
! polyglot download morph2.en

[polyglot_data] Downloading package morph2.en to
[polyglot_data]     /root/polyglot_data...


In [ ]:
words = ["preprocessing", "processor", "invaluable", "thankful", "crossed"]
for w in words:
  w = Word(w, language="en")
  print("{:<20}{}".format(w, w.morphemes))

preprocessing       ['pre', 'process', 'ing']
processor           ['process', 'or']
invaluable          ['in', 'valuable']
thankful            ['thank', 'ful']
crossed             ['cross', 'ed']


Если с английским работает хорошо, совсем не значит, что с другими обещанными языками тоже.

In [ ]:
! polyglot download morph2.ru

[polyglot_data] Downloading package morph2.ru to
[polyglot_data]     /root/polyglot_data...


In [ ]:
words = ["хождение", "предложение", "защищающихся", "привет", "вынуть", "поле"]
for w in words:
  w = Word(w, language="ru")
  print("{:<20}{}".format(w, w.morphemes))

хождение            ['хождение']
предложение         ['предложен', 'ие']
защищающихся        ['защи', 'щ', 'а', 'ющихся']
привет              ['при', 'вет']
вынуть              ['вы', 'нуть']
поле                ['поле']


С русским хорошо работает [XMorphy](https://github.com/alesapin/XMorphy)